# Data Ingestion: Motion Sense

## Initial Setup

In [1]:
#Import python modules
import boto3
import configparser
import json
import pandas as pd
import random
import time


In [ ]:
#Load configurations 
#Ask Riu for access key information and update config file before running
%run load_config.py

In [ ]:
#Define firehose object
firehose = boto3.client(
    'firehose',
    region_name = "us-east-2",
    aws_access_key_id = access_key_id,
    aws_secret_access_key = secret_access_key)

## Prepare Data for Streaming

In [ ]:
#Read data
r_path = "../../../Motion Sense/Data/"
csv_name = "df_all_data.csv"

df = pd.read_csv(r_path + csv_name, low_memory = False)

In [ ]:
#Column names
column_names = ["tick_num","attitude.roll","attitude.pitch","attitude.yaw","gravity.x","gravity.y","gravity.z","rotationRate.x","rotationRate.y","rotationRate.z","userAcceleration.x","userAcceleration.y","userAcceleration.z","test_type","subject_id","test_trial_number","time_since_start","time_series_data","attitude","gravity","rotationRate","userAcceleration","weight","height","age","gender"]

#Convert specified columns from int64 to string
df["tick_num"] = df["tick_num"].astype(str)
df["test_trial_number"] = df["test_trial_number"].astype(str)
df["weight"] = df["weight"].astype(str)
df["height"] = df["height"].astype(str)
df["age"] = df["age"].astype(str)
df["gender"] = df["gender"].astype(str)

In [ ]:
#Keep only test data for simulating streaming
#df = (filter data to only keep test data)

## Simulate Streaming

In [ ]:
#Define number of rows in data
nrow = df.shape[0]

In [ ]:
#Simulate data streaming
for i in range(nrow):
    response = firehose.put_record(
        DeliveryStreamName = DeliveryStreamName,
        Record = {
            "Data": json.dumps(
                {
                    f"{}".format(column_names[0]): final_df_full.at[df.index[i],f"{}".format(column_names[0])],
                    f"{}".format(column_names[1]): final_df_full.at[df.index[i],f"{}".format(column_names[1])],
                    f"{}".format(column_names[2]): final_df_full.at[df.index[i],f"{}".format(column_names[2])],
                    f"{}".format(column_names[3]): final_df_full.at[df.index[i],f"{}".format(column_names[3])],
                    f"{}".format(column_names[4]): final_df_full.at[df.index[i],f"{}".format(column_names[4])],
                    f"{}".format(column_names[5]): final_df_full.at[df.index[i],f"{}".format(column_names[5])],
                    f"{}".format(column_names[6]): final_df_full.at[df.index[i],f"{}".format(column_names[6])],
                    f"{}".format(column_names[7]): final_df_full.at[df.index[i],f"{}".format(column_names[7])],
                    f"{}".format(column_names[8]): final_df_full.at[df.index[i],f"{}".format(column_names[8])],
                    f"{}".format(column_names[9]): final_df_full.at[df.index[i],f"{}".format(column_names[9])],
                    f"{}".format(column_names[10]): final_df_full.at[df.index[i],f"{}".format(column_names[10])],
                    f"{}".format(column_names[11]): final_df_full.at[df.index[i],f"{}".format(column_names[11])],
                    f"{}".format(column_names[0]): final_df_full.at[df.index[i],f"{}".format(column_names[12])],
                    f"{}".format(column_names[1]): final_df_full.at[df.index[i],f"{}".format(column_names[13])],
                    f"{}".format(column_names[2]): final_df_full.at[df.index[i],f"{}".format(column_names[14])],
                    f"{}".format(column_names[3]): final_df_full.at[df.index[i],f"{}".format(column_names[15])],
                    f"{}".format(column_names[4]): final_df_full.at[df.index[i],f"{}".format(column_names[16])],
                    f"{}".format(column_names[5]): final_df_full.at[df.index[i],f"{}".format(column_names[17])],
                    f"{}".format(column_names[6]): final_df_full.at[df.index[i],f"{}".format(column_names[18])],
                    f"{}".format(column_names[7]): final_df_full.at[df.index[i],f"{}".format(column_names[19])],
                    f"{}".format(column_names[8]): final_df_full.at[df.index[i],f"{}".format(column_names[20])],
                    f"{}".format(column_names[9]): final_df_full.at[df.index[i],f"{}".format(column_names[21])],
                    f"{}".format(column_names[10]): final_df_full.at[df.index[i],f"{}".format(column_names[22])],
                    f"{}".format(column_names[11]): final_df_full.at[df.index[i],f"{}".format(column_names[23])],
                    f"{}".format(column_names[10]): final_df_full.at[df.index[i],f"{}".format(column_names[24])],
                    f"{}".format(column_names[11]): final_df_full.at[df.index[i],f"{}".format(column_names[25])]
                }
            )
        }
    )
    time.sleep(round(random.uniform(0.01, 0.15), 3))